# $\star\star$ Currently in development.  This example doesn't work $\star\star$
Some parts of `pygsti` are works-in-progress. Here, we investigate how to do the task of "model selection" within GST, essentially answering the question "Can we do a better job of modeling the experiment by changing the assumptions within GST?".

## Testing variable-gateset-dimension GST with model selection

###     Setup

In [1]:
from __future__ import print_function

In [2]:
import pygsti
from pygsti.construction import std1Q_XYI

In [3]:
#Load gateset and some string lists
gs_target = std1Q_XYI.gs_target
fiducialList = std1Q_XYI.fiducials
germList = std1Q_XYI.germs
specs = pygsti.construction.build_spam_specs(fiducialList)
expList = [1,2,4]


In [4]:
#Create some testing gate string lists
lgstList = pygsti.construction.list_lgst_gatestrings(specs, gs_target.gates.keys())
lsgstLists = [ lgstList[:] ]
for exp in expList:
    gsList = pygsti.construction.create_gatestring_list(
                "f0+germ*exp+f1", f0=fiducialList, f1=fiducialList,
                germ=germList, exp=exp, order=['germ','f0','f1'])
    lsgstLists.append( lsgstLists[-1] +  gsList )
    
dsList = pygsti.remove_duplicates( lsgstLists[-1] )

In [6]:
#Test on fake data by depolarizing target set, increasing its dimension,
# and adding leakage to the gates into the new dimension.

gs_dataGen4 = gs_target.depolarize(gate_noise=0.1)
gs_dataGen5 = gs_dataGen4.increase_dimension(5)
leakGate = pygsti.construction.build_gate( [2,1],[('Q0',),('L0',)] , "LX(pi/4.0,0,2)","gm") # X(pi,Q0)*LX(pi,0,2)

gs_dataGen5['Gx'] = pygsti.objects.compose( gs_dataGen5['Gx'], leakGate, 'gm')
gs_dataGen5['Gy'] = pygsti.objects.compose( gs_dataGen5['Gy'], leakGate, 'gm')
print(gs_dataGen5.gates.keys())

#Some debugging...
#NOTE: with LX(pi,0,2) above, dim 5 test will choose a dimension 3 gateset, which may be sensible
#       looking at the gate matrices in this case... but maybe LX(pi,...) is faulty?
#print(gs_dataGen4)
#print(gs_dataGen5)

#Jmx = GST.JOps.jamiolkowski_iso(gs_dataGen4['Gx'])
#Jmx = GST.JOps.jamiolkowski_iso(gs_dataGen5['Gx'],dimOrStateSpaceDims=[2,1])
#print("J = \n",Jmx)
#print("evals = ",eigvals(Jmx))

dsFake4 = pygsti.construction.simulate_data(gs_dataGen4, dsList, nSamples=1000000, sampleError="binomial", seed=1234)
dsFake5 = pygsti.construction.simulate_data(gs_dataGen5, dsList, nSamples=1000000, sampleError="binomial", seed=1234)

odict_keys(['Gi', 'Gx', 'Gy'])


AssertionError: 

In [6]:
print("Number of gates                        = ",len(gs_target.gates.keys()))
print("Number of fiducials                    =",len(fiducialList))
print("Maximum length for a gate string in ds =",max(map(len,dsList)))
print("Number of LGST strings                 = ",len(lgstList))
print("Number of LSGST strings                = ",map(len,lsgstLists))

Number of gates                        =  3
Number of fiducials                    = 6
Maximum length for a gate string in ds = 30
Number of LGST strings                 =  92
Number of LSGST strings                =  <map object at 0x10a021ba8>


### Test using dimension-4 fake data

In [7]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
# NOTE: with nSamples less than 1M (100K, 10K, 1K) this routine will choose a higher-than-4 dimensional gateset
ds = dsFake4
gs_lgst4 = pygsti.run_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=3)
gs_lgst6 = pygsti.run_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=6, verbosity=3)

#Print chi^2 of 4-dim and 6-dim estimates
chiSq4 = pygsti.chi2(ds, gs_lgst4, lgstList, minProbClipForWeighting=1e-4)
chiSq6 = pygsti.chi2(ds, gs_lgst6, lgstList, minProbClipForWeighting=1e-4)
print("LGST dim=4 chiSq = ", chiSq4)
print("LGST dim=6 chiSq = ", chiSq6)

# Least squares GST with model selection
gs_lsgst = pygsti.do_iterative_mc2gst_with_model_selection(ds, gs_lgst4, 1, lsgstLists, verbosity=2,
                                                           minProbClipForWeighting=1e-3, probClipInterval=(-1e5,1e5))

--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  3.00262981818
  0.825857216429
  0.671539150077
  0.664532662258
  0.000838614351546
  0.000292355239716
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  3.0
  1.0
  1.0
  1.0
  2.46365884775e-16
  9.80956846773e-17
  
    Resulting gate set:
    
    rho0 =    0.7072  -0.0214   0.0213   0.7510
    
    
    E0 =    0.6847   0.0028  -0.0026  -0.6446
    
    
    Gi = 
       1.0002        0   0.0003  -0.0001
      -0.0042   0.9000   0.0005  -0.0004
       0.0032  -0.0018   0.8995  -0.0009
      -0.0030   0.0002   0.0013   0.8999
    
    
    Gx = 
       1.0001  -0.0002        0        0
      -0.0037   0.8982   0.0055   0.0005
       0.0006  -0.0014  -0.0052  -1.0001
      -0.0624   0.0070   0.8104   0.0065
    
    
    Gy = 
       1.0003   0.0002        0  -0.0003
      -0.0001  -0.0066   0.0002   0.9989
       0.0041   0.0069   0.8989        0
      -0.0625  -0.8098  -0.0065   0.0

In [8]:
print(gs_lsgst)

rho0 =    0.7072  -0.0217   0.0216   0.7511


E0 =    0.6849   0.0022  -0.0021  -0.6447


Gi = 
   1.0000        0        0        0
  -0.0035   0.9001   0.0001   0.0003
   0.0035        0   0.8998   0.0002
  -0.0034  -0.0001   0.0002   0.9000


Gx = 
   1.0000        0        0        0
  -0.0037   0.8998   0.0060   0.0004
   0.0004  -0.0004  -0.0063  -0.9999
  -0.0625   0.0067   0.8101   0.0064


Gy = 
   1.0000        0        0        0
  -0.0002  -0.0065        0   0.9994
   0.0038   0.0072   0.9000  -0.0009
  -0.0624  -0.8105  -0.0059   0.0066





### Test using dimension-5 fake data

In [9]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
ds = dsFake5
gs_lgst4 = pygsti.run_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=3)
gs_lgst6 = pygsti.run_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=6, verbosity=3)

#Print chi^2 of 4-dim and 6-dim estimates
chiSq4 = pygsti.chi2(ds, gs_lgst4, lgstList, minProbClipForWeighting=1e-2)
chiSq6 = pygsti.chi2(ds, gs_lgst6, lgstList, minProbClipForWeighting=1e-2)
print("LGST dim=4 chiSq = ", chiSq4)
print("LGST dim=6 chiSq = ", chiSq6)

# Least squares GST with model selection
gs_lsgst = pygsti.do_iterative_mc2gst_with_model_selection(ds, gs_lgst4, 1, lsgstLists, verbosity=2, minProbClipForWeighting=1e-3, probClipInterval=(-1e5,1e5), useFreqWeightedChiSq=False, regularizeFactor=1.0, check=False, check_jacobian=False)

--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  3.56525557364
  0.755465956932
  0.496239591347
  0.487203220172
  0.00691003018424
  0.000321987001111
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  3.0
  1.0
  1.0
  1.0
  2.46365884775e-16
  9.80956846773e-17
  
    Resulting gate set:
    
    rho0 =    0.6961  -0.0307   0.0425   0.7832
    
    
    E0 =    0.7554   0.0111  -0.0141  -0.6685
    
    
    Gi = 
       0.9997  -0.0005   0.0007  -0.0063
      -0.0064   0.8998   0.0007  -0.0020
       0.0037  -0.0023   0.8991  -0.0002
      -0.0080   0.0006   0.0017   0.9024
    
    
    Gx = 
       1.0223   0.0007   0.0476  -0.0050
      -0.0039   0.8292   0.0241  -0.0040
       0.0070   0.0263  -0.0403  -1.0042
      -0.1197   0.0279   0.6526   0.0271
    
    
    Gy = 
       1.0221  -0.0478  -0.0009  -0.0034
      -0.0182  -0.0538   0.0319   1.0079
       0.0003   0.0110   0.8295  -0.0148
      -0.1260  -0.6490  -0.0304   0.03

In [10]:
print(gs_lsgst)

rho0 =    0.7072  -0.0141   0.0218   0.7481  -0.0373


E0 =    0.7007   0.0431  -0.0421  -0.6482   0.2446


Gi = 
   1.0000        0        0        0        0
  -0.0042   0.8991   0.0007   0.0014  -0.0073
   0.0023   0.0004   0.8992  -0.0011   0.0078
  -0.0091  -0.0002   0.0004   0.9002  -0.0006
  -0.0002   0.0092  -0.0090  -0.0138   1.0004


Gx = 
   1.0000        0   0.0001  -0.0001        0
  -0.0210   0.8322   0.0094  -0.0114   0.0043
   0.0199   0.0222  -0.0175  -1.0220  -0.0412
  -0.1081   0.0314   0.6711   0.0272  -0.0476
   0.1734  -0.0049   0.0221  -0.0089   0.7705


Gy = 
   1.0000  -0.0001        0  -0.0002        0
  -0.0364  -0.0359   0.0230   1.0195   0.0415
   0.0164  -0.0023   0.8315  -0.0122  -0.0053
  -0.1186  -0.6737  -0.0266   0.0465  -0.0455
   0.1732  -0.0227   0.0049  -0.0071   0.7705



